## Load libraries

In [18]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
sys.path.append("../libs")
sys.path.append("../")
from definitions import ROOT_DIR
import utils as utils

# Set pandas options
# Not use scientific notation and use two decimal places and use comma as thousand separator
pd.options.display.float_format = '{:,.2f}'.format


# ETL

## 1. Load dataset

In [19]:
# Load dataset from data_files/internet.xlsx - sheet: 'Ingresos '
df = utils.get_xls_sheet_data('/data_files/internet.xlsx', 'Accesos Por Tecnología')
df.head().style.format(thousands=',')

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,"2,024",2,Buenos Aires,"214,055.000000","2,722,466.000000","1,849,476.000000","138,638.000000","64,745.000000","4,989,380.000000"
1,"2,024",2,Capital Federal,"54,102.000000","1,144,781.000000","230,402.000000","4,493.000000","29,821.000000","1,463,599.000000"
2,"2,024",2,Catamarca,"4,951.000000","10,303.000000","58,355.000000","1,384.000000",81.000000,"75,074.000000"
3,"2,024",2,Chaco,"9,448.000000","57,935.000000","68,944.000000","8,407.000000","2,358.000000","147,092.000000"
4,"2,024",2,Chubut,"25,955.000000","80,704.000000","26,516.000000","31,118.000000","9,930.000000","174,223.000000"


## 2. Handling missing values

### Look for missing values in all cells

In [20]:
#Find missing values
df.isnull().sum()

Año             1
Trimestre       1
Provincia       2
ADSL            2
Cablemodem      2
Fibra óptica    2
Wireless        2
Otros           2
Total           2
dtype: int64

#### There are 16 missed values, next we will analyse them and decide how to handle them

In [21]:
df[df.isnull().any(axis=1)]

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008,*,Los datos provinciales no coinciden a nivel na...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### All missed values are concentrated in 2 rows, the row index 1007 correspond to "Buenos Aires" T1 year 2024. And 1008 are empty in the original datasource so we can drop it.

In [22]:
#fill missing values with 0 for 1007 index

df.loc[1007] = [2024, 1, 'Buenos Aires', 218723, 2760083, 1817035, 138331, 66344, 5000500]

In [23]:
#remove missing values
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1008 entries, 0 to 1007
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           1008 non-null   object 
 1   Trimestre     1008 non-null   object 
 2   Provincia     1008 non-null   object 
 3   ADSL          1008 non-null   float64
 4   Cablemodem    1008 non-null   float64
 5   Fibra óptica  1008 non-null   float64
 6   Wireless      1008 non-null   float64
 7   Otros         1008 non-null   float64
 8   Total         1008 non-null   float64
dtypes: float64(6), object(3)
memory usage: 78.8+ KB


## 3. Look for Duplicates

### Find duplicates for complete rows

In [24]:
#Find duplicates by complete row
df.duplicated().sum()

0

#### There's no complete duplicated rows

### Fin duplicated rows for year and quarter ('Trimestre') and state ('Provincia')

In [25]:
#Find duplicated rows by row, for year and quarter
df.duplicated(subset=['Año', 'Trimestre', 'Provincia']).sum()

0

#### There's no duplicated rows for year, quarter and state

## 4. Data Types

### We will review the data types for each column.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1008 entries, 0 to 1007
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           1008 non-null   object 
 1   Trimestre     1008 non-null   object 
 2   Provincia     1008 non-null   object 
 3   ADSL          1008 non-null   float64
 4   Cablemodem    1008 non-null   float64
 5   Fibra óptica  1008 non-null   float64
 6   Wireless      1008 non-null   float64
 7   Otros         1008 non-null   float64
 8   Total         1008 non-null   float64
dtypes: float64(6), object(3)
memory usage: 78.8+ KB


#### The 'Año', 'Trimestre' and 'Provincia' columns have an object data type, and have strings values. We should change the data type to string.

In [27]:
# Review unique data
df['Año'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, '2019 *', 2018, 2017, 2016,
       2015, 2014], dtype=object)

#### For quarters 1 to 3 for year 2019 we have the value 2019*, this means the data in not realiable because the values where misinformed from the source. I think the best solution is to copy the 4th quarter values for the year 2019.

In [28]:
#select Año = '2019' and Trimestre = '1'
ok_2019 = df[df['Año'] == 2019].copy()
ok_2019['Trimestre'] = '3'
df = pd.concat([df, ok_2019])

ok_2019['Trimestre'] = '2'
df = pd.concat([df, ok_2019])

ok_2019['Trimestre'] = '1'
df = pd.concat([df, ok_2019])

# drop rows where 'Año' = '2019 *'
df = df.drop(df[df['Año'] == '2019 *'].index)

df.groupby(['Año','Trimestre'])['Provincia'].agg(['count'])


count
Año  Trimestre       
2014 1             24
     2             24
     3             24
     4             24
2015 1             24
     2             24
     3             24
     4             24
2016 1             24
     2             24
     3             24
     4             24
2017 1             24
     2             24
     3             24
     4             24
2018 1             24
     2             24
     3             24
     4             24
2019 4             24
     1             24
     2             24
     3             24
2020 1             24
     2             24
     3             24
     4             24
2021 1             24
     2             24
     3             24
     4             24
2022 1             24
     2             24
     3             24
     4             24
2023 1             24
     2             24
     3             24
     4             24
2024 1             24
     2             24

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1008 entries, 0 to 454
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           1008 non-null   object 
 1   Trimestre     1008 non-null   object 
 2   Provincia     1008 non-null   object 
 3   ADSL          1008 non-null   float64
 4   Cablemodem    1008 non-null   float64
 5   Fibra óptica  1008 non-null   float64
 6   Wireless      1008 non-null   float64
 7   Otros         1008 non-null   float64
 8   Total         1008 non-null   float64
dtypes: float64(6), object(3)
memory usage: 78.8+ KB


#### We needs to change the data type to string for 'Provincia' and integer for 'Año' and 'Trimestre' columns.

In [30]:
df['Año'] = df['Año'].astype('int64')
df['Trimestre'] = df['Trimestre'].astype('int64')
df['Provincia'] = df['Provincia'].astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1008 entries, 0 to 454
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           1008 non-null   int64  
 1   Trimestre     1008 non-null   int64  
 2   Provincia     1008 non-null   string 
 3   ADSL          1008 non-null   float64
 4   Cablemodem    1008 non-null   float64
 5   Fibra óptica  1008 non-null   float64
 6   Wireless      1008 non-null   float64
 7   Otros         1008 non-null   float64
 8   Total         1008 non-null   float64
dtypes: float64(6), int64(2), string(1)
memory usage: 78.8 KB


## 5. Finding outliers

In [31]:
#Finding outliers
df.describe()

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
count,"1,008.00","1,008.00","1,008.00","1,008.00","1,008.00","1,008.00","1,008.00","1,008.00"
mean,"2,018.76",2.45,"110,525.34","185,490.05","51,616.00","12,763.99","6,500.71","366,896.07"
std,3.04,1.12,"229,440.01","454,732.85","186,501.73","21,380.60","11,277.12","788,194.42"
min,"2,014.00",1.00,"2,230.00",0.00,0.00,0.00,2.00,"12,557.00"
25%,"2,016.00",1.00,"19,521.25","11,015.00",355.00,814.00,341.50,"62,418.25"
50%,"2,019.00",2.00,"43,967.50","40,892.50","3,224.50","5,245.00","1,901.00","113,834.50"
75%,"2,021.00",3.00,"88,040.50","77,159.25","31,092.75","14,531.00","7,622.00","196,158.50"
max,"2,024.00",4.00,"1,586,343.00","2,797,700.00","1,849,476.00","138,638.00","73,415.00","5,011,620.00"


#### In the statistics, we can see there's no outliers values for the column year ("Año"), because the maximum value is 2024 and the data is historical. For the quarter ("Trimestre") column the minimum an maximum values are correct, 1 and 4. We have 0 values in dome technologies, this maybe means that technolofy isn't available in that province in that year, i.e. "Cable Modem" in "La Rioja" province from 2014 to 2017. No outlies detected.

In [32]:
df.groupby(['Año','Trimestre'])['Provincia'].agg(['count'])

count
Año  Trimestre       
2014 1             24
     2             24
     3             24
     4             24
2015 1             24
     2             24
     3             24
     4             24
2016 1             24
     2             24
     3             24
     4             24
2017 1             24
     2             24
     3             24
     4             24
2018 1             24
     2             24
     3             24
     4             24
2019 1             24
     2             24
     3             24
     4             24
2020 1             24
     2             24
     3             24
     4             24
2021 1             24
     2             24
     3             24
     4             24
2022 1             24
     2             24
     3             24
     4             24
2023 1             24
     2             24
     3             24
     4             24
2024 1             24
     2             24

#### All pairs year-quarter are present, there's no missing data for each of the 24 provinces.

## New columns

### Create a new column with the year and quarter

In [33]:
#Create a new column with the quarter and year
df['Periodo'] = (df['Año'].astype(str) + 'T' + df['Trimestre'].astype(str)).astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1008 entries, 0 to 454
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           1008 non-null   int64  
 1   Trimestre     1008 non-null   int64  
 2   Provincia     1008 non-null   string 
 3   ADSL          1008 non-null   float64
 4   Cablemodem    1008 non-null   float64
 5   Fibra óptica  1008 non-null   float64
 6   Wireless      1008 non-null   float64
 7   Otros         1008 non-null   float64
 8   Total         1008 non-null   float64
 9   Periodo       1008 non-null   string 
dtypes: float64(6), int64(2), string(2)
memory usage: 86.6 KB


# Save dataset

In [34]:
df=df.reset_index()
df.to_parquet(ROOT_DIR + '/data_files/accesos_por_tecnologia_por_provincia_clean.parquet')